In [ ]:
# 이 Python 3 environment는 많은 유용한 분석 libraires의 설치와 함께합니다. 
# This Python 3 environment comes with many helpful analytics libraries installed

# 이는 kaggle/python docker image로 정의되어 있습니다 : https://github.com/kaggle/docker-python
# It is defined by the kaggle/python docker image: https://github.com/kaggle/docker-python

# 예시로, load해서 사용해볼만한 몇가지의 유용한 packages들을 보여드리겠습니다.
# For example, here's several helpful packages to load in 

import numpy as np # linear algebra (선형대수학)
import pandas as pd # data processing, CSV file I/O (e.g. pd.read_csv) (데이터 처리, CSV file의 I/O)

# Input data files는 "../input/" directory를 통해 제공됩니다.
# Input data files are available in the "../input/" directory.

# 예를 들면, 이것을 실행시키면 (run을 누르거나, Shift+Enter를 누르세요) input directory에 있는 file들을 보여줄 것입니다.
# For example, running this (by clicking run or pressing Shift+Enter) will list the files in the input directory

from subprocess import check_output
print(check_output(["ls", "../input"]).decode("utf8"))

# 현재 directory에 쓴 모든 results들은 output 형태로 저장될 것입니다.
# Any results you write to the current directory are saved as output.

이 kernel은 Keras를 위해 CNN을 구현하고 싶어하는 초심자들을 위한 것입니다. 이 kernel을 이용하면, 당신은 좋은 점수를 받기를 기대할 수 있을 뿐만 아니라 keras에 대해서도 배울 수 있습니다.
Keras는 model을 initialize하고 우리가 원하는 layer를 계속 쌓을 수 있는 간단한 frameworks 입니다. 이것은 deep neural networks를 아주 쉽게 만들 수 있도록 도와줍니다.

(This kernel is specifically is for Beginners who want's to experiment building CNN using Keras. By using this kernel, you can expect to get good score and also learn keras. 
Keras is simple frameworks where we can initialize the model and keep stacking the layers we want. It makes building deep neural networks very easy.)

In [ ]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
from os.path import join as opj
from matplotlib import pyplot as plt
from mpl_toolkits.mplot3d import Axes3D
import pylab
plt.rcParams['figure.figsize'] = 10, 10
%matplotlib inline

In [ ]:
#Load the data.
train = pd.read_json("../input/train.json")

In [ ]:
test = pd.read_json("../input/test.json")

#데이터에 대한 기본 설명
#Intro about the Data.

Sentinel 인공위성은 지구에서 680Km 떨어진 지점에 위치해 있습니다. 특정 입사각에서 신호들을 보내, 돌아오는 신호를 기록합니다. 기본적으로 반사되어 되돌아온 신호는 "backscatter"이라고 불립니다. 우리에게 주어진 데이터는 backscatter 변수로 전통적인 backscatter 변수의 형태를 띕니다. 그 형태는 다음과 같습니다 : 
(Sentinet -1 sat is at about 680 Km above earth. Sending pulses of signals at a particular angle of incidence and then recoding it back. Basically those reflected signals are called backscatter. The data we have been given is backscatter coefficient which is the conventional form of backscatter coefficient given by:)

$σo (dB) = βo (dB) + 10log10 [ sin(ip) / sin (ic)] $

1. ip = 특정 pixel에서의 입사각
2. ic = 이미지 중심에서의 입사각
3. K = 상수

(where
1. ip=is angle of incidence for a particular pixel
2. 'ic ' is angle of incidence for center of the image
3. K =constant.)

데이터에는 $σo$가 직접적으로 제공됩니다.
We have been given $σo$ directly in the data.

###$σo$의 특징
기본적으로 σo는 signal이 어느 표면에서 산란되었는지에 따라 다릅니다. 예를 들어, 어떤 특정한 입사각에 대해 다음과 같은 다른 형태를 나타냅니다 : 
(Basically σo varies with the surface on which the signal is scattered from. For example, for a particular angle of incidence, it varies like:)

*             WATER...........           SETTLEMENTS........           AGRICULTURE...........          BARREN........

1.**HH:**     -27.001   ................                     2.70252       .................                -12.7952        ................    -17.25790909

2.**HV: **      -28.035      ................            -20.2665             ..................          -21.4471       .................     -20.019

위에서 보다싶이, HH 구성 요소들은 변화가 크지만, HV는 그렇지 않습니다.
**저는 배에서 산란한 데이터는 갖고 있지 않습니다. 하지만 금속 물체임에 따라, 이것은 얼음 물체와는 다른 형태로 변화해야 합니다.**
(As you can see, the HH component varies a lot but HV doesn't.
**I don't have the data for scatter from ship, but being a metal object, it should vary differently as compared to ice object.**)

###HH와 HV는 무엇입니까?
이 Sentinel 인공위성은 RISTSAT 인공위성 (인도의 원격탐사 인공위성)과 같으며, 그것들은 ping들을 수평 편광(H polarization)으로만 보냅니다 **(수직 편광(V polarization)으로는 보내지 않습니다.)**.  그 H-pings들은 산란되며,  물체들은 그것들의 편광상태를 변경시키게 되고,  H와 V의 혼합으로 반환됩니다.
**Sentinal이 오직 H-송신기 만을 가짐에 따라, 돌아오는 신호는 HH와 HV 형태만 존재합니다.**. 왜 VV가 제공되지 않았는지 물어보지 마십시오(왜냐하면 Sentinel은 V-ping 송신기를 갖고 있지 않습니다).

이제 feature를 봅시다. 이 데모 코드의 목적은 RGB와 비슷한 3-channel 데이터를 만들기 위해, 두개의 band(HH, HV)를 추출한 후 그것들을 평균을 낸 것을 3rd channel으로 사용할 것입니다.

(Ok, so this Sentinel Settalite is equivalent to RISTSAT(an Indian remote sensing Sat) and they only Transmit pings in H polarization, **AND NOT IN V polarization**.  Those H-pings gets scattered, objects change their polarization and returns as a mix of H and V.
**Since Sentinel has only H-transmitter, return signals are of the form of HH and HV only**. Don't ask why VV is not given(because Sentinel don't have V-ping transmitter).

Now coming to features, for the purpose of this demo code, I am extracting all two bands and taking avg of them as 3rd channel to create a 3-channel RGB equivalent. )


In [ ]:
# training 데이터를 만듭니다.
# HH, HV, HH와 HV의 평균 총 3개의 band를 만듭니다. 
#Generate the training data
#Create 3 bands having HH, HV and avg of both
X_band_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_1"]])
X_band_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in train["band_2"]])
X_train = np.concatenate([X_band_1[:, :, :, np.newaxis], X_band_2[:, :, :, np.newaxis],((X_band_1+X_band_2)/2)[:, :, :, np.newaxis]], axis=-1)

In [ ]:
#빙산을 봅시다
#Take a look at a iceberg
import plotly.offline as py
import plotly.graph_objs as go
py.init_notebook_mode(connected=True)
def plotmy3d(c, name):
    print(c.shape)
    data = [
        go.Surface(
            z=c
        )
    ]
    layout = go.Layout(
        title=name,
        autosize=False,
        width=700,
        height=700,
        margin=dict(
            l=65,
            r=50,
            b=65,
            t=90
        )
    )
    fig = go.Figure(data=data, layout=layout)
    py.iplot(fig)
plotmy3d(X_band_1[12,:,:], 'iceberg')

멋진 빙산이 보이네요. 레이더 데이터에서 빙산의 모양은 여기서 보이는 것과  같은 산 모양을 나타낼 것입니다. 이것이 실제 이미지가 아닌 레이더에서 산란된 결과임에 따라, 이것처럼 뾰족한 봉우리와 왜곡된 모양으로 나타날 것입니다. 배의 모양은 점, 혹은 연장된 점처럼 보일 것입니다. 여기에서 둘의 구조상의 차이점이 나타나고, 우리는 CNN을 이용하여 이 차이점을 활용할 수 있습니다. 우리가 레이더로부터 얻은 backscatter를 이용하여 합성 사진을 만들 수 있다면, 많은 도움이 될 것입니다.  
(That's a cool looking iceberg we have. Remember, in radar data, the shape of the iceberg is going to be like a mountain as shown in here. Since this is not a actual image but scatter from radar, the shape is going to have peaks and distortions like these. The shape of the ship is going to be like a point, may be like a elongated point. From here the structural differences arise and we can exploit those differences using a CNN. It would be helpful if we can create composite images using the backscatter from radar.)

In [ ]:
plotmy3d(X_band_1[14,:,:], 'Ship')

이것이 배입니다, 연결되어 있는 점같이 생겼네요. 배 모양을 시각화하기엔 해상도가 부족합니다. 그것을 도와주기위해 CNN이 존재합니다. 배-빙산 분류에 관련된 논문이 다음과 같이 존재합니다:
http://elib.dlr.de/99079/2/2016_BENTES_Frost_Velotto_Tings_EUSAR_FP.pdf
그런데 그들의 데이터는 훨씬 좋은 해상도를 갖고 있기 때문에 그들이 이용한 CNN은 여기에 맞지 않을 것 같습니다.

That's a ship, looks like a elongated point. We don't have much resolution in images to visualize the shape of the ship. However CNN is here to help. There are few papers on ship iceberg classification like this:
http://elib.dlr.de/99079/2/2016_BENTES_Frost_Velotto_Tings_EUSAR_FP.pdf
However their data have much better resolution so I don't  feel that the CNN they used would be suitable here.

다시 돌아가서 Keras로 CNN을 만들어 봅시다. 다른 framework보다 훨씬 좋습니다. 당신도 좋아할 거에요.
(Get back to building a CNN using Keras. Much better frameworks then others. You will enjoy for sure.)

In [ ]:
#Import Keras.
from matplotlib import pyplot
from keras.preprocessing.image import ImageDataGenerator
from keras.models import Sequential
from keras.layers import Conv2D, MaxPooling2D, Dense, Dropout, Input, Flatten, Activation
from keras.layers import GlobalMaxPooling2D
from keras.layers.normalization import BatchNormalization
from keras.layers.merge import Concatenate
from keras.models import Model
from keras import initializers
from keras.optimizers import Adam
from keras.callbacks import ModelCheckpoint, Callback, EarlyStopping

In [ ]:
#define our model
def getModel():
    #Building the model
    gmodel=Sequential()
    #Conv Layer 1
    gmodel.add(Conv2D(64, kernel_size=(3, 3),activation='relu', input_shape=(75, 75, 3)))
    gmodel.add(MaxPooling2D(pool_size=(3, 3), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Conv Layer 2
    gmodel.add(Conv2D(128, kernel_size=(3, 3), activation='relu' ))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Conv Layer 3
    gmodel.add(Conv2D(128, kernel_size=(3, 3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Conv Layer 4
    gmodel.add(Conv2D(64, kernel_size=(3, 3), activation='relu'))
    gmodel.add(MaxPooling2D(pool_size=(2, 2), strides=(2, 2)))
    gmodel.add(Dropout(0.2))

    #Flatten the data for upcoming dense layers
    gmodel.add(Flatten())

    #Dense Layers
    gmodel.add(Dense(512))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))

    #Dense Layer 2
    gmodel.add(Dense(256))
    gmodel.add(Activation('relu'))
    gmodel.add(Dropout(0.2))

    #Sigmoid Layer
    gmodel.add(Dense(1))
    gmodel.add(Activation('sigmoid'))

    mypotim=Adam(lr=0.001, beta_1=0.9, beta_2=0.999, epsilon=1e-08, decay=0.0)
    gmodel.compile(loss='binary_crossentropy',
                  optimizer=mypotim,
                  metrics=['accuracy'])
    gmodel.summary()
    return gmodel


def get_callbacks(filepath, patience=2):
    es = EarlyStopping('val_loss', patience=patience, mode="min")
    msave = ModelCheckpoint(filepath, save_best_only=True)
    return [es, msave]
file_path = ".model_weights.hdf5"
callbacks = get_callbacks(filepath=file_path, patience=5)


In [ ]:
target_train=train['is_iceberg']
X_train_cv, X_valid, y_train_cv, y_valid = train_test_split(X_train, target_train, random_state=1, train_size=0.75)

In [ ]:
#Without denoising, core features.
import os
gmodel=getModel()
gmodel.fit(X_train_cv, y_train_cv,
          batch_size=24,
          epochs=50,
          verbose=1,
          validation_data=(X_valid, y_valid),
          callbacks=callbacks)

###Though the score may be different here,  it works good on LB, I got 0.210 score.

In [ ]:
gmodel.load_weights(filepath=file_path)
score = gmodel.evaluate(X_valid, y_valid, verbose=1)
print('Test loss:', score[0])
print('Test accuracy:', score[1])

In [ ]:

X_band_test_1=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_1"]])
X_band_test_2=np.array([np.array(band).astype(np.float32).reshape(75, 75) for band in test["band_2"]])
X_test = np.concatenate([X_band_test_1[:, :, :, np.newaxis]
                          , X_band_test_2[:, :, :, np.newaxis]
                         , ((X_band_test_1+X_band_test_2)/2)[:, :, :, np.newaxis]], axis=-1)
predicted_test=gmodel.predict_proba(X_test)

In [ ]:
submission = pd.DataFrame()
submission['id']=test['id']
submission['is_iceberg']=predicted_test.reshape((predicted_test.shape[0]))
submission.to_csv('sub.csv', index=False)

#### 결론
점수를 올리기 위해, 나는 Speckle filtering, Indicence angle normalization, 그리고 다른 preprocessing을 사용해 봤지만 잘 작동하지 않았습니다. 당신도 해보고 결과를 내볼수는 있겠지만, 나의 경우엔 그것들이 좋은 결과를 내지는 않았다.

이 kernel을 이용해서 상위 10명 안에 들지는 못할것이니, 좋은 정보를 하나 주겠습니다. test dataset에 8000개의 이미지가 포함되어있고, 우리는 그것을 활용해 볼 수 있습니다. 에측 정확도를 올리기 위해 pseudo labeling을 할 수 있습니다. 여기 관련 글이 있습니다:
https://towardsdatascience.com/simple-explanation-of-semi-supervised-learning-and-pseudo-labeling-c2218e8c769b

이 kernel이 좋았으면 Upvote해주세요.

#### Conclusion
To increase the score, I have tried Speckle filtering, Indicence angle normalization and other preprocessing and they don't seems to work.  You may try and see but for me they are not giving any good results.

You can't be on top-10 using this kernel, so here is one beautiful peice of information. The test dataset contain 8000 images, We can exploit this. We can do pseudo labelling to increase the predictions. Here is the article related to that:
https://towardsdatascience.com/simple-explanation-of-semi-supervised-learning-and-pseudo-labeling-c2218e8c769b

Upvote if you liked this kernel.